In [ ]:
# | default_exp _components.docs_dependencies

# Install docs dependencies

In [ ]:
# | export

import asyncio
from tempfile import TemporaryDirectory
import subprocess
import shutil

from fastkafka._components.logger import get_logger

In [ ]:
import os
from contextlib import contextmanager

import pytest

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
# | export


async def _check_npx(required_major_version: int = 9) -> None:
    if shutil.which("npx") is not None:
        cmd = "npx --version"
        proc = await asyncio.create_subprocess_shell(
            cmd, stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
        )
        output, _ = await proc.communicate()
        major_version = int(output.decode("UTF-8").split(".")[0])
        if major_version < required_major_version:
            raise RuntimeError(
                f"Found installed npx major version: {major_version}, required npx major version: {required_major_version}. To use documentation features of FastKafka, please update npx"
            ) 
    else:
        raise RuntimeError(
            f"npx not found, to use documentation generation features of FastKafka, you must have npx >= {required_major_version} installed"
        )

In [ ]:
@contextmanager
def _clean_path():
    path = os.environ["PATH"]
    try:
        os.environ["PATH"] = ""
        yield
    finally:
        os.environ["PATH"] = path

In [ ]:
await _check_npx()

In [ ]:
with _clean_path():
    with pytest.raises(RuntimeError) as e:
        await _check_npx()
        
assert e.value.args[0] == "npx not found, to use documentation generation features of FastKafka, you must have npx >= 9 installed"

In [ ]:
with pytest.raises(RuntimeError) as e:
    await _check_npx(required_major_version=999)
        
assert e.value.args[0] == "Found installed npx major version: 9, required npx major version: 999. To use documentation features of FastKafka, please update npx", e.value.args[0]

In [ ]:
# | export

def _install_docs_deps() -> None:
    with TemporaryDirectory() as d:
        cmd = [
            "npx",
            "-y",
            "-p",
            "@asyncapi/generator",
            "ag",
            "https://raw.githubusercontent.com/asyncapi/asyncapi/master/examples/simple.yml",
            "@asyncapi/html-template",
            "-o",
            d,
        ]

        p = subprocess.run(  # nosec: B603 subprocess call - check for execution of untrusted input.
            cmd, stderr=subprocess.STDOUT, stdout=subprocess.PIPE
        )
        if p.returncode == 0:
            logger.info("AsyncAPI generator installed")
        else:
            logger.error("AsyncAPI generator NOT installed!")
            logger.info(f"Output of '$ {' '.join(cmd)}'{p.stdout.decode()}")
            raise ValueError(
                f"AsyncAPI generator NOT installed, used '$ {' '.join(cmd)}'{p.stdout.decode()}"
            )

In [ ]:
_install_docs_deps()